In [ ]:
# Exercise 3.1
import re
import Levenshtein

def moderate_content(text):
    tier1_words = ["terrorism", "kill", "murder"]
    tier2_phrases = ["make money fast", "buy followers", "click this link"]
    tier3_words = ["stupid", "idiot", "hate", "dumb"]

    censored_text = text.lower()

    # Stage 1.1: severe checks
    for word in tier1_words:
        if re.search(rf"\b{word}\b", censored_text):
            return "[content removed due to severe violation]", 5.0
    for phrase in tier2_phrases:
        if re.search(re.escape(phrase), censored_text):
            return "[content removed due to spam/scam policy]", 5.0

    # Stage 1.2: scored filtering
    score = 0.0

    # Tier 3 + fuzzy check
    words_in_text = re.findall(r"\b\w+\b", censored_text)
    for word in words_in_text:
        for bad_word in tier3_words:
            similarity = 1 - Levenshtein.distance(word, bad_word) / max(len(word), len(bad_word))
            if similarity >= 0.8:
                censored_text = re.sub(rf"\b{word}\b", "*" * len(word), censored_text, flags=re.IGNORECASE)
                score += 2.0
                break

    # Link detection
    links = re.findall(r"(https?://\S+|www\.\S+)", censored_text)
    if links:
        censored_text = re.sub(r"(https?://\S+|www\.\S+)", "[link removed]", censored_text)
        score += 2.0 * len(links)

    # Excessive capitalization
    letters = [c for c in text if c.isalpha()]
    if len(letters) > 15:
        upper_ratio = sum(1 for c in letters if c.isupper()) / len(letters)
        if upper_ratio > 0.7:
            score += 0.5

    return censored_text, min(score, 5.0)

text = "MAKE MONEY FAST!! This is stupiid: https://spam.com"
print(moderate_content(text))


: 

In [ ]:
import sqlite3
from datetime import datetime

def get_last_date(cur):
    cur.execute("""
        SELECT MAX(latest_date) FROM (
            SELECT MAX(created_at) AS latest_date FROM users
            UNION
            SELECT MAX(created_at) AS latest_date FROM posts
            UNION
            SELECT MAX(created_at) AS latest_date FROM comments
        )
    """)
    result = cur.fetchone()[0]
    return datetime.strptime(result, "%Y-%m-%d %H:%M:%S")


def user_risk_analysis(cur, user_id, today):

    cur.execute("SELECT profile, created_at FROM users WHERE id = ?", (user_id,))
    row = cur.fetchone()
    if not row:
        return None
    profile, created_at = row

    account_age_days = (today - datetime.strptime(created_at, "%Y-%m-%d %H:%M:%S")).days

    profile_score = moderate_content(profile)[1] if profile else 0.0

    cur.execute("SELECT content FROM posts WHERE user_id = ?", (user_id,))
    posts = [row[0] for row in cur.fetchall()]
    post_scores = [moderate_content(p)[1] for p in posts] if posts else []
    avg_post_score = sum(post_scores) / len(post_scores) if post_scores else 0.0

    cur.execute("SELECT content FROM comments WHERE user_id = ?", (user_id,))
    comments = [row[0] for row in cur.fetchall()]
    comment_scores = [moderate_content(c)[1] for c in comments] if comments else []
    avg_comment_score = sum(comment_scores) / len(comment_scores) if comment_scores else 0.0

    content_risk_score = (profile_score * 1) + (avg_post_score * 3) + (avg_comment_score * 1)

    if account_age_days < 7:
        user_risk_score = content_risk_score * 1.5
    elif account_age_days < 30:
        user_risk_score = content_risk_score * 1.2
    else:
        user_risk_score = content_risk_score

    cur.execute("""SELECT reaction_type, COUNT(1) FROM reactions WHERE user_id = ? GROUP BY reaction_type""", (user_id,))
    reactions = dict(cur.fetchall())
    neg_reacts = reactions.get("angry", 0) + reactions.get("sad", 0)
    total_reacts = sum(reactions.values()) if reactions else 0

    if total_reacts > 0:
        negativity_ratio = neg_reacts / total_reacts
        user_risk_score += negativity_ratio

    return min(user_risk_score, 5.0)



conn = sqlite3.connect("/content/database.sqlite")
cur = conn.cursor()
cur.execute("SELECT id, username FROM users")
users = cur.fetchall()

today = get_last_date(cur)

results = []
for user_id, username in users:
    score = user_risk_analysis(cur, user_id, today)
    if score is not None:
        results.append((user_id, username, score))

conn.close()

top5 = sorted(results, key=lambda x: x[2], reverse=True)[:5]
for id, user, score in top5:
    print(f"{id} - {user}: {score}")


In [ ]:
from collections import Counter

def extract_keywords(text):
    if not text:
        return []
    words = re.findall(r"#\w+|\b\w+\b", text.lower())
    words = [w for w in words if len(w) > 2]
    return words


def recommend(cur, user_id):
    cur.execute("""
        SELECT p.id, p.content
        FROM posts p
        JOIN reactions r ON p.id = r.post_id
        WHERE r.user_id = ? AND r.reaction_type IN ('like', 'love', 'wow')
    """, (user_id,))
    liked_posts_data = cur.fetchall()

    liked_posts = [p[0] for p in liked_posts_data]
    keyword_freq = Counter()

    for id, content in liked_posts_data:
        keyword_freq.update(extract_keywords(content))


    cur.execute("SELECT followed_id FROM follows WHERE follower_id = ?", (user_id,))
    followed_users = [r[0] for r in cur.fetchall()]

    followed_posts = []
    if followed_users:
        cur.execute(f"""
            SELECT id, content
            FROM posts
            WHERE user_id IN ({','.join(['?'] * len(followed_users))})
        """, followed_users)
        followed_posts = cur.fetchall()

    exclude_ids = ",".join(map(str, liked_posts)) if liked_posts else "0"
    cur.execute(f"SELECT id, content FROM posts WHERE id NOT IN ({exclude_ids})")
    all_posts = cur.fetchall()

    scored_posts = []
    for post_id, content in all_posts:
        post_words = extract_keywords(content)
        similarity = sum(keyword_freq[w] for w in post_words if w in keyword_freq)
        scored_posts.append((post_id, similarity))

    for id, content in followed_posts:
        scored_posts.append((id, 1.0))

    scored_posts = sorted(scored_posts, key=lambda x: x[1], reverse=True)
    top_5 = scored_posts[:5]

    recommended = []
    for id, score in top_5:
        cur.execute("SELECT content FROM posts WHERE id = ?", (id,))
        text = cur.fetchone()
        if text:
            recommended.append((id, text[0], round(score, 2)))

    return recommended


conn = sqlite3.connect("/content/database.sqlite")
cur = conn.cursor()

recs = recommend(cur, user_id=42)
for id, text, score in recs:
    print(f"Post {id} (score {score}): {text[:80]}...")
